In [1]:
%matplotlib inline

In [2]:
import json
import csv

import pandas as pd
import numpy as np

In [3]:
import seaborn as sns
sns.set_style("whitegrid")
import matplotlib
matplotlib.rcParams['figure.figsize'] = (15.0, 8.0)

In [4]:
from icecream import ic

In [5]:
region_name = "Томск"
region_data_file = "tomskaia-oblast.geojson"  # from https://dtp-stat.ru/opendata/

In [6]:
def get_dtp_by_region():
    def check_region(item):
        return item["properties"]["region"] == region_name

    with open(region_data_file, "rt") as f:
        dtp = json.load(f)

    res = list(filter(check_region, dtp["features"]))
    print(f'всего ДТП в регионе "{region_name}": {len(res)}')
    return res
dtp_tomsk = get_dtp_by_region()

всего ДТП в регионе "Томск": 2318


In [7]:
def feature_extractor(it):
    res = {
        "id": it["properties"]["id"],
        "datetime": it["properties"]["datetime"],
        "lat": it["properties"]["point"]["lat"],
        "lon": it["properties"]["point"]["long"],
        "severity": it["properties"]["severity"],
        "address": it["properties"]["address"],
        "dead_count": it["properties"]["dead_count"],
        "injured_count": it["properties"]["injured_count"],
    }
    return res
df_dtp = pd.DataFrame(map(feature_extractor, dtp_tomsk))
df_dtp.datetime = df_dtp.datetime.astype(np.datetime64)
df_dtp.head()

,id,datetime,lat,lon,severity,address,dead_count,injured_count
0,2528452,2020-12-20 23:02:00,56.503054,84.958584,Легкий,"г Томск, ул Дальне-Ключевская, 25Б",0,1
1,2503256,2020-10-07 20:15:00,56.576648,85.105934,С погибшими,"г Томск, Кузовлево-Светлый, 6 км",1,0
2,2539023,2021-01-31 08:20:00,56.496890,84.968401,Легкий,"г Томск, ул Яковлева, 9",0,1
3,2539024,2021-01-30 14:30:00,56.462004,84.989151,Легкий,None,0,1
4,2539025,2021-01-29 21:36:00,56.480713,85.033740,Тяжёлый,"г Томск, ул Писемского, 8",0,1


In [8]:
repairs = []
with open("00_fragments.csv", "rt") as f:
    reader = csv.reader(f)
    header = next(reader)
    print("Header columns:", header)
    for row in reader:
        repairs.append(dict(zip(header,row)))
df_repairs = pd.DataFrame(repairs)

df_repairs[["work_start", "work_end"]] = df_repairs["Срок выполнения работ на объекте"]\
    .str.split(" ‒ ", n=1, expand=True).astype(np.datetime64)

df_repairs.head()

Header columns: ['Год работ', 'fragment_header', 'fragment_title', 'comments', 'id', 'Статус', 'Срок выполнения работ на объекте', 'Тип работ']


,Год работ,fragment_header,fragment_title,comments,id,Статус,Срок выполнения работ на объекте,Тип работ,work_start,work_end
0,2021,Ремонт,пр.Коммунистического от ул.Свердлова до ул.Лео...,,e8809a70-2216-4bee-922b-154d73ae493f,Запланирован,25.04.2021 ‒ 01.09.2021,Ремонт,2021-04-25,2021-01-09
1,2017,Ремонт покрытия проезжей части,Автодорога «ул.Кирова»,,70ee27cb-ab73-4efb-bcc2-93c0b5a4b85c,Завершен,01.06.2017 ‒ 31.07.2017,Ремонт покрытия проезжей части,2017-01-06,2017-07-31
2,2020,Ремонт,ул.Крупской: от просп.Коммунистического до ул....,,4a52c0b2-7805-4bf1-b9b7-d090263c28de,Завершен,25.04.2020 ‒ 31.08.2020,Ремонт,2020-04-25,2020-08-31
3,2018,Ремонт покрытия проезжей части,ул. Войкова,,56402bec-90dc-4184-bbda-07e4a7191180,Завершен,01.05.2018 ‒ 01.09.2018,Ремонт покрытия проезжей части,2018-01-05,2018-01-09
4,2018,Ремонт покрытия проезжей части,ул. Набережная р. Ушайки,,25d81872-36a6-4066-8132-58f14f969e50,Завершен,01.05.2018 ‒ 01.09.2018,Ремонт покрытия проезжей части,2018-01-05,2018-01-09


In [9]:
with open("01_dtp_to_fragments.json", "rt") as f:
    dtp_to_repair = json.load(f)
df_dtp_to_repair = pd.DataFrame(dtp_to_repair)

In [10]:
# for every repair match all related DTPs
df_merged = df_repairs\
    .merge(df_dtp_to_repair, left_on="id", right_on="fragment")\
    .merge(df_dtp, left_on="dtp_id", right_on="id", suffixes=("_repair", "_dtp"))

In [11]:
df_merged.severity = df_merged.severity.astype("category")
df_merged.severity.cat.set_categories(["Легкий", "Тяжёлый", "С погибшими"], inplace=True)  # sorting order

In [12]:
df_merged

,Год работ,fragment_header,fragment_title,comments,id_repair,Статус,Срок выполнения работ на объекте,Тип работ,work_start,work_end,dtp_id,fragment,id_dtp,datetime,lat,lon,severity,address,dead_count,injured_count
0,2018,Ремонт покрытия проезжей части,ул. Войкова,,56402bec-90dc-4184-bbda-07e4a7191180,Завершен,01.05.2018 ‒ 01.09.2018,Ремонт покрытия проезжей части,2018-01-05,2018-01-09,2515609,56402bec-90dc-4184-bbda-07e4a7191180,2515609,2020-11-11 23:32:00,56.502616,84.940903,Тяжёлый,"г Томск, пер Заозерный, 3а",0,1
1,2018,Ремонт покрытия проезжей части,ул. Набережная р. Ушайки,,25d81872-36a6-4066-8132-58f14f969e50,Завершен,01.05.2018 ‒ 01.09.2018,Ремонт покрытия проезжей части,2018-01-05,2018-01-09,2281358,25d81872-36a6-4066-8132-58f14f969e50,2281358,2018-09-30 01:00:00,56.484985,84.950205,Легкий,None,0,1
2,2018,Ремонт покрытия проезжей части,ул. Набережная р. Ушайки,,25d81872-36a6-4066-8132-58f14f969e50,Завершен,01.05.2018 ‒ 01.09.2018,Ремонт покрытия проезжей части,2018-01-05,2018-01-09,2282285,25d81872-36a6-4066-8132-58f14f969e50,2282285,2015-07-16 15:30:00,56.486667,84.950556,Тяжёлый,"г Томск, ул Набережная реки Ушайки, 4а",0,1
3,2017,Ремонт покрытия проезжей части,Автомобильная дорога Томск - Мирный - Меженино...,,cf61a288-bf68-4df7-9209-ce56dda01abf,Завершен,19.06.2017 ‒ 10.10.2017,Ремонт покрытия проезжей части,2017-06-19,2017-10-10,2279868,cf61a288-bf68-4df7-9209-ce56dda01abf,2279868,2017-06-18 03:25:00,56.452800,85.030600,Тяжёлый,"г Томск, ул Богдана Хмельницкого, 60",0,1
4,2017,Ремонт покрытия проезжей части,Автомобильная дорога Томск - Мирный - Меженино...,,cf61a288-bf68-4df7-9209-ce56dda01abf,Завершен,19.06.2017 ‒ 10.10.2017,Ремонт покрытия проезжей части,2017-06-19,2017-10-10,2281089,cf61a288-bf68-4df7-9209-ce56dda01abf,2281089,2016-08-25 22:05:00,56.451900,85.035000,Легкий,"г Томск, ул Степная 1-я, 12а",0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1426,2017,Ремонт покрытия проезжей части,Чекистский тракт,,c1604843-8c3a-46f6-a7e0-6d6ff5841a68,Завершен,18.04.2017 ‒ 05.10.2017,Ремонт покрытия проезжей части,2017-04-18,2017-05-10,2282870,c1604843-8c3a-46f6-a7e0-6d6ff5841a68,2282870,2018-01-09 14:30:00,56.548300,84.971700,Тяжёлый,"г Томск, тракт Кузовлевский, 24",0,3
1427,2017,Ремонт покрытия проезжей части,Чекистский тракт,,c1604843-8c3a-46f6-a7e0-6d6ff5841a68,Завершен,18.04.2017 ‒ 05.10.2017,Ремонт покрытия проезжей части,2017-04-18,2017-05-10,2282978,c1604843-8c3a-46f6-a7e0-6d6ff5841a68,2282978,2016-10-28 11:30:00,56.547500,84.971100,Тяжёлый,"г Томск, тракт Кузовлевский, 24",0,1
1428,2017,Ремонт покрытия проезжей части,Чекистский тракт,,c1604843-8c3a-46f6-a7e0-6d6ff5841a68,Завершен,18.04.2017 ‒ 05.10.2017,Ремонт покрытия проезжей части,2017-04-18,2017-05-10,2279075,c1604843-8c3a-46f6-a7e0-6d6ff5841a68,2279075,2018-06-18 07:50:00,56.538648,84.965129,Легкий,"г Томск, тракт Кузовлевский, 31",0,1
1429,2017,Ремонт покрытия проезжей части,Чекистский тракт,,c1604843-8c3a-46f6-a7e0-6d6ff5841a68,Завершен,18.04.2017 ‒ 05.10.2017,Ремонт покрытия проезжей части,2017-04-18,2017-05-10,2528444,c1604843-8c3a-46f6-a7e0-6d6ff5841a68,2528444,2020-12-31 20:28:00,56.537719,84.965333,Тяжёлый,"г Томск, ул Кутузова, 1А",0,1


In [13]:
# assign "before", "during", or "after" based on DTP datetime and repair begun/finished 
df_merged["repair_relation"] = np.where(df_merged.datetime < df_merged.work_start, "before",
                                        np.where(df_merged.datetime > df_merged.work_start, "after",
                                                 "during"))
# df_merged.repair_relation = df_merged.repair_relation.astype("category")
# df_merged.repair_relation.cat.set_categories(["before", "during", "after"], inplace=True)  # sorting order

In [14]:
# Count period lengths
dtp_first_day = np.datetime64("2015-01-01")
dtp_last_day = np.datetime64("2021-02-28")

df_merged["total_days"] = np.where(df_merged.repair_relation=="before", df_merged.work_start-dtp_first_day,
                                   np.where(df_merged.repair_relation=="after", dtp_last_day-df_merged.work_end,
                                   df_merged.work_end-df_merged.work_start))
# converting to ints
df_merged["total_days"] = df_merged.total_days.dt.days

In [15]:
dtp_2020 = df_merged[df_merged["Год работ"] == "2020"]
print("Всего ДТП, на дорогах отремонтированных в 2020-ом году: ", dtp_2020.shape[0])
print("Из них до ремонта (2015-2020): ", dtp_2020[dtp_2020.repair_relation=="before"].shape[0])
print("Из них после ремонта (2020-начало 2021): ", dtp_2020[dtp_2020.repair_relation=="after"].shape[0])
dtp_2020[dtp_2020.repair_relation=="after"][[
 'fragment_title',
 'Срок выполнения работ на объекте',
 'datetime',
 'severity',
 'address',
 'dead_count',
 'injured_count']]

Всего ДТП, на дорогах отремонтированных в 2020-ом году:  170
Из них до ремонта (2015-2020):  151
Из них после ремонта (2020-начало 2021):  19


,fragment_title,Срок выполнения работ на объекте,datetime,severity,address,dead_count,injured_count
101,"ул. 79 Гв. Дивизии от ул. 79 Гв. Дивизии, 1 до...",15.05.2020 ‒ 30.08.2020,2021-01-20 10:30:00,С погибшими,"г Томск, ул 79 Гвардейской Дивизии, 3",1,1
120,"ул. 79 Гв. Дивизии от ул. 79 Гв. Дивизии, 1 до...",15.05.2020 ‒ 30.08.2020,2020-09-03 21:00:00,Тяжёлый,"г Томск, ул 79 Гвардейской Дивизии, 2 Б",0,1
122,"ул. 79 Гв. Дивизии от ул. 79 Гв. Дивизии, 1 до...",15.05.2020 ‒ 30.08.2020,2020-12-06 00:10:00,Легкий,"г Томск, ул 79 Гвардейской Дивизии, 6",0,1
431,"ул. Пролетарская от ул. Пролетарская, 2 до пр....",15.05.2020 ‒ 30.08.2020,2021-01-13 18:55:00,Тяжёлый,"г Томск, ул Пролетарская, 18",0,1
436,"ул. Пролетарская от ул. Пролетарская, 2 до пр....",15.05.2020 ‒ 30.08.2020,2020-11-01 19:10:00,Легкий,"г Томск, пр-кт Ленина, 196",0,1
468,"ул. Елизаровых от ул. Елизаровых, 11Б до ул. Ш...",15.05.2020 ‒ 30.08.2020,2021-01-30 14:30:00,Легкий,None,0,1
469,"ул. Елизаровых от ул. Елизаровых, 11Б до ул. Ш...",15.05.2020 ‒ 30.08.2020,2021-02-13 17:50:00,Легкий,"г Томск, ул Елизаровых, 22",0,1
470,"ул. Елизаровых от ул. Елизаровых, 11Б до ул. Ш...",15.05.2020 ‒ 30.08.2020,2020-10-12 07:30:00,Тяжёлый,"г Томск, ул Елизаровых, 50",0,1
491,"ул. Елизаровых от ул. Елизаровых, 11Б до ул. Ш...",15.05.2020 ‒ 30.08.2020,2020-06-27 10:55:00,Тяжёлый,"г Томск, ул Елизаровых, 2а",0,1
558,"ул. Косарева от ул. Косарева, 1/1 до ул. Косар...",15.05.2020 ‒ 30.08.2020,2020-09-09 13:50:00,Тяжёлый,"г Томск, ул Косарева, 33",0,1


In [16]:
# remove 2021 year, because it holds future (planned) repairs
df_grouped = df_merged[(df_merged["Год работ"] != "2021") & (df_merged.repair_relation != "during")]\
        .groupby(["fragment_title", "repair_relation"])\
        .agg(totals=("id_repair", "count"), days=("total_days", "first"), year=("Год работ", "first"))

df_grouped["ratio"] = df_grouped.totals/df_grouped.days*365

df_grouped = df_grouped.unstack()

df_grouped.columns = [f"{a}_{b}" for a, b in df_grouped.columns]
df_grouped = df_grouped.drop(columns=["year_before"]).rename(columns={"year_after": "year"})
df_grouped

,totals_after,totals_before,days_after,days_before,year,ratio_after,ratio_before
fragment_title,,,,,,,
"Автодорога ""Томск-ТНХК""",3.0,2.0,1280.0,737.0,2017,0.855469,0.990502
Автомобильная дорога Кузовлево - Светлый (км 0+000 - км 6+730),13.0,6.0,1237.0,837.0,2017,3.835893,2.616487
"Автомобильная дорога Михайловка - Александровское - Итатка с подъездом к с.Малиновка (км 0+000 - км 50+513), подъезд к с.Малиновка (км 0+000 - км 2+055)",5.0,NaN,1511.0,NaN,2017,1.207809,NaN
Автомобильная дорога Томск - Аэропорт (км 2+500 - км 20+185),16.0,15.0,1146.0,1099.0,2018,5.095986,4.981802
Автомобильная дорога Томск - Каргала - Колпашево (км 5+976 - км 51+241),11.0,9.0,1217.0,856.0,2017,3.299096,3.837617
...,...,...,...,...,...,...,...
"ул. Усова от ул. Усова, 11а до ул.Усова, 66",1.0,7.0,507.0,1595.0,2019,0.719921,1.601881
ул. Учебная,20.0,9.0,1277.0,766.0,2017,5.716523,4.288512
ул. Ф. Лыткина,1.0,5.0,1146.0,1100.0,2018,0.318499,1.659091


In [17]:
df_grouped["change"] = df_grouped.ratio_after/df_grouped.ratio_before

df_to_save = df_grouped[(df_grouped.ratio_after > 2) | (df_grouped.ratio_before > 2)].dropna().sort_values("change")

df_to_save

,totals_after,totals_before,days_after,days_before,year,ratio_after,ratio_before,change
fragment_title,,,,,,,,
Коммунальный мост с подходами (проезжая часть),1.0,7.0,1277.0,929.0,2017,0.285826,2.750269,0.103927
Автомобильная дорога Томск - Мариинск (км0+000 - км 76+737),5.0,12.0,1217.0,930.0,2017,1.499589,4.709677,0.318406
ул. Советская,3.0,8.0,1146.0,1100.0,2018,0.955497,2.654545,0.359948
ул. Смирнова от ул.Смирнова 1 до ул.Кольцевой проезд 52,6.0,22.0,568.0,1595.0,2019,3.855634,5.034483,0.765845
ул. Гоголя,6.0,5.0,1277.0,838.0,2017,1.714957,2.177804,0.787471
ул. Р. Люксембург,20.0,16.0,1247.0,838.0,2017,5.854050,6.968974,0.840016
Автомобильная дорога Томск - Каргала - Колпашево (км 5+976 - км 51+241),11.0,9.0,1217.0,856.0,2017,3.299096,3.837617,0.859673
ул. Тверская,9.0,10.0,1146.0,1100.0,2018,2.866492,3.318182,0.863874
Московский тракт,11.0,12.0,1146.0,1100.0,2018,3.503490,3.981818,0.879872


In [18]:
columns_renaming = {
    "totals_after": "Кол-во ДТП после ремонта",
    "totals_before": "Кол-во ДТП до ремонта",
    "days_after": "Дней наблюдения после ремонта",
    "days_before": "Дней наблюдения до ремонта",
    "year": "Год выполнения ремонта",
    "ratio_after": "Частота ДТП (случаев в год) после ремонта",
    "ratio_before": "Частота ДТП (случаев в год) до ремонта",
    "change": "Изменение частоты ДТП"
}
df_to_save.index.name = "Отремонтированный участок"
df_to_save.rename(columns=columns_renaming).to_excel("dtp_rate_changes.xlsx")